# Bidiagonalization with Householder Reflectors: Golub-Kahan Method

### Motivation

The bidiagonalization method Golub and Kahan proposed in 1965 sought to eliminate computational difficulties in calculating the singular value decomposition of matrices [1]. 

Singular value decomposition (SVD) is a powerful tool in machine learning. The SVD algorithm factors a matrix: $A$ into one matrix with **orthogonal columns**, one with **orthogonal rows**, and a diagonal matrix which contains the **singular values** of the original matrix [2]. 

SVD is used in dimensionality reduction with numerous applications including: topic modeling, recommender systems, and image filtering. A great example matching customers to movies can be found at [3].


The resulting singular values: $\Sigma$ are relative importance factors of the original matrix.

SVD of matrix $A$:

$$A = U\Sigma V^T$$

Bidiagonalization is the first step in many matrix decomposition algorithms where the original matrix: $A$ is **ill-conditioned** or **rank deficient**; often this is the case in image blurring scenarios [4].

### Methodology

The Golub-Kahan method tranforms matrix $A$ into a **bidiagonal matrix: $J$**. 

Using **Householder reflectors**, applied alternating from the left and from the right, matrix entries are zeroed leaving behind a bidiagonal of non-zero values. The resulting bidiagonal matrix $J$ can then be diagonalized to solve for the singular values. 

Instead of the SVD equation above, the Golub-Kahan method decomposes the original matrix $A$ into the matrices: $P\,$, $Q\,$, $J$ below, with $J$ as the final bidiagonal matrix. $P$ and $Q$ are **unitary matrices** . The decomposition of matrix $A$ and the equation rewritten in terms of $J$ for clarity with the diagram to follow:

$$A = P\,J\,Q^{T}\quad\quad\quad J = P^{T}A\,Q$$

Given a matrix $A$ below, where non-zero entries are represented by dots:


<img src="images/matrix1.png"style="width: 28%"/>

Note that $A$ is a dense matrix. There is another bidiagonalization technique for sparse matrices; the Golub-Kahan-Lanczos method. This sparse method also referred to as the Lanczos bidiagonalization [5].


The diagram below demonstrates Golub-Kahan bidiagonalization [6], [4].
The basics of what's going on in the diagram:

   1. choose the first column, marked by $\mathbf{X}\,$, to compute our Householder vector
   
   2. then apply the Householder reflector (using the Householder vector) to the left of the matrix $A$
   
   3. choose the first row, marked by $\mathbf{X}\,$, to compute our Householder vector
   
   4. apply the Householder reflector (using the Householder vector) to the right of the matrix $A$
   
   5. repeat steps 1-4 until there are no more rows and/or columns to choose
   
   
   
   

<img src="images/Bidiag.png"style="width: 85%"/>

The final bidiagonalized matrix is $J$, while $P$ and $Q$ are made up of the matrix products of the alternating Householder reflectors applied to the left and right of $A$: $U_{1}$, $U_{2}$,$U_{3}$, ..., and $V_{1}$,$V_{2}$, ..., respectively. 

It is important to note that the length of the alternating rows and columns we use to compute the Householder vectors get smaller with each step as we approach the final matrix $J$. The corresponding Householder reflectors shrink in dimension as well.

There are 2 equivalent versions of bidiagonalization; **upper** and **lower**. This diagram shows the upper bidiagonalization, with the second diagonal located **above** the main diagonal. Upper and lower diagonal matrices are equivalent (need citation). Bidiagonalization is performed on matrices $A$ with shape ($m\;x\;n$) where $m \geq n$.

### Householder Vectors and Reflectors

Now we know Householder reflectors are used in bidiagonalization, QR factorization, and SVD. But... 

What are Householder vectors and Householder reflectors?? 

Remember a matrix times a vector is essentially just shifting the vector; rotating and/or stretching it. A **Householder reflector is a matrix that shifts a vector to be non-zero in its first element and zero everywhere else while maintaining the original vector's Euclidean norm**, or distance from the origin. 

So, given a column or row vector: $z$ from our original matrix: $A$, we'd like to chose our **Householder reflector: $P$**, such that: 

$$Pz = \big\|z\big\|_{2}e_{1}$$

To gain some further intuition about how the **Householder reflector:** $P$, is caluclated we'll look at this geometrically with a 2-dimensional example [7]:


<img src="images/reflector1.png"style="width: 35%"/>

We have taken the original vector: $z$ applied the reflector: $P$ such that now $Pz$ lies on the x axis.
We can also rewrite the reflector equation to express it as a sum of 2 vectors:

$$Pz = \big\|z\big\|_{2}e_{1} = z + v$$

where **$v$** is the **Householder vector** and by definition: $$v = \big\|z\big\|_{2}e_{1} - z$$

<img src="images/reflector2.png"style="width: 35%"/>

The term reflector is used because we say that $z$ has been reflected about the mirror onto the axis. It is important to remember this is not a projection; it is a reflection. $Pz$ maintains the same length as $z$. The way we have set the problem up geometrically, our mirror is a perpendicular bisector of $v$. We can now solve for $P$ using the geometric relationships between the vectors: $z$, $v$, and our mirror.

<img src="images/reflector3.png"style="width: 35%"/>

We start by finding the length of the vector: $w$. Adding $w$ to $z$ would get us to the mirror:

$$\big\|w\big\| = \big\|z\big\|cos\,\theta \quad\quad cos\,\theta = \frac{-z^{T}v}{\big\|z\big\|\big\|v\big\|}\quad\Longrightarrow\quad \big\|w\big\|= \frac{-z^{T}v}{\big\|v\big\|}$$

Since $w$ has the same direction as $v$, its direction is:

$$\hat{w} = \frac{v}{\big\|v\big\|}$$

Putting length and direction together we get:

$$w =\hat{w}\big\|w\big\|\quad\Longrightarrow\quad -\frac{v\,(z^{T}v)}{\big\|v\big\|\big\|v\big\|}\quad\Longrightarrow\quad -\frac{v\,(z^{T}v)}{v^{T}v}$$

But again, this vector $w$ will only get us to the mirror. We want the vector that will get us to the reflection $Pz$. All we need to do is multiply by 2:

$$Pz = z - 2\frac{v\,(z^{T}v)}{v^{T}v}$$ 

Rearranging the scalar $(z^{T}v)$ to $(v^{T}z)$ and solving for our Householder reflector P:

$$P = I - \frac{2}{v^{T}v}vv^{T}$$ 

### Algorithm 

This is a brief example algorithm; i.e. it does not do error handling. Our Householder vector is $v$. To see how the algorithm follows the explanation above I added print statements in the function. We can see how the bidiagonal is being created step by step.

Also note, I added a helper function to eliminate issues from round-off error.

In [1]:
import numpy as np

In [2]:
np.set_printoptions(suppress=True)

In [3]:
def set_lowVal_zero(X):
    low_values_indices = abs(X) < 9e-15   # where values are low
    X[low_values_indices] = 0             # all low values set to 0
    return X

In [4]:
A = np.array([[4, 3, 0, 2, 5], [2, 1, 2, 1, 7], [4, 4, 0, 3, 8], [5, 6, 1, 3, 1]])
J = A.copy(); J

array([[4, 3, 0, 2, 5],
       [2, 1, 2, 1, 7],
       [4, 4, 0, 3, 8],
       [5, 6, 1, 3, 1]])

In [5]:
def Householder(x, i):
    alpha = -np.sign(x[i]) * np.linalg.norm(x)
    e = np.zeros(len(x)); e[i] = 1.0
    
    v = (x - alpha * e)
    w = v / np.linalg.norm(v)
    P = np.eye(len(x)) - 2 * np.outer(w, w.T)
    
    return P

In [6]:
def Golub_Kahan(X):
    col = X.shape[1]
    row = X.shape[0]
    
    J = X.copy()
    count = 1
    for i in range(col - 2):
        # column
        h = np.zeros(len(J[:, i]))
        h[i:] = J[i:, i]
        P = Householder(h, i)
        J = set_lowVal_zero(P @ J)
        print(J, '\n')

        # row
        h = np.zeros(len(J[i, :]))
        h[i+1:] = J[i, i+1:] 
        Q = Householder(h, i+1)
        J = set_lowVal_zero(J @ Q)
        print(J, '\n')
        
    return J

In [7]:
Golub_Kahan(A)

[[-7.81024968 -7.6822128  -1.15233192 -4.73736456 -9.09061848]
 [ 0.         -0.80897324  1.80485901 -0.14093516  4.61383225]
 [ 0.          0.38205352 -0.39028198  0.71812968  3.22766449]
 [ 0.          1.47756691  0.51214752  0.1476621  -4.96541939]] 

[[ -7.81024968  12.86181284   0.           0.           0.        ]
 [  0.          -2.88761934   1.68826598  -0.62026207   3.69404277]
 [  0.          -2.73902218  -0.56534578  -0.00157705   1.84660564]
 [  0.           2.52670888   0.57099479   0.38958976  -4.50117983]] 

[[ -7.81024968  12.86181284   0.           0.           0.        ]
 [  0.           4.71432346  -0.39959864   0.58964563  -5.74802607]
 [  0.           0.          -1.31761499   0.43435941  -1.55542416]
 [  0.           0.           1.2649524   -0.01255541  -1.36285588]] 

[[ -7.81024968  12.86181284   0.           0.           0.        ]
 [  0.           4.71432346   5.79199143   0.           0.        ]
 [  0.           0.           1.67874107   0.1490065    1.2

array([[ -7.81024968,  12.86181284,   0.        ,   0.        ,   0.        ],
       [  0.        ,   4.71432346,   5.79199143,   0.        ,   0.        ],
       [  0.        ,   0.        ,  -2.10137347,   0.19450319,   0.        ],
       [  0.        ,   0.        ,   0.        ,   1.55389757,  -0.9662093 ]])

### Terminology

- **bidiagonal matrix**: a matrix with non-zero entries along the main diagonal and either the diagonal above (upper) or below (lower) with zeroes elsewhere


- **orthogonal columns/ rows**: columns/ rows are perpendicular to one another


- **orthogonal matrix**: when a matrix $A$ times its transpose $A^{T}$ is the identity matrix $I$


- **singular values**: the relative importance of factors in SVD. Also, the square roots of the eigenvalues of the symmetric nxn matrix $A^{T}A$ listed with their values in decreasing order


- **ill-conditioned**: when the approximation of the solutions $x$ to the equation $Ax = b$ is above a level of inaccuracy, due to the values of the matrix $A$, i.e. when a small error in $b$ may cause a large error in $x$


- **rank deficient**: for an mxn matrix $A$, when the number of linearly independent rows is less than m


- **unitary matrix**: similar to an orthogonal matrix, but holds for complex matrices, where the product of a matrix $A$ and its complex conjugate transpose $A^*$ is the identity matrix $I$

### References

[1] G. H. Golub, W. Kahan, "Calculating the Singular Values and Pseudoinverse of a Matrix," SIAM J. Numer. Anal. 2, 205–224 (1965).


[2] R. Thomas, MSAN Numerical Linear Algebra Course Notes: Topic Modeling with NMF and SVD. University of San Francisco (2017). https://github.com/fastai/numerical-linear-algebra


[3] Video Tutorials All in One, "Singular Value Decomposition" Stanford University, YouTube, 13 April (2016).
https://www.youtube.com/watch?v=P5mlg91as1c. 


[4] M. Plesinger, "Some Remarks on Bidiagonalization and Its Implementation," PhD Conference: Institute of Computer Science, Prague (2005).
http://www2.cs.cas.cz/mweb/download/publi/Ples2006.pdf


[5] A. Bjork, "A Bidiagonalization Algorithm for Solving Large and Sparse Ill-posed Systems of Linear Equations," BIT Numerical Mathematics. 28(3), 659-670 (1988).
https://link.springer.com/article/10.1007/BF01941141



[6] G. Fasshauer, 444/577 Handouts and Worksheets: Chapter 12. Illinois Institue of Technology. http://www.math.iit.edu/~fass/477577_Chapter_12.pdf


[7] T. Driscoll, "Householder QR" MATH426, YouTube, 23 September (2015).
https://www.youtube.com/watch?v=d-yPM-bxREs



[8] G. H. Golub, C. F. Van Loan, Matrix Computations 3rd edition, The John Hopkins University Press, Baltimore, (1996).



[9] L. N. Trefethen and D. Bau, Numerical Linear Algebra, SIAM (1997).